# Data Encoder
Uses an autoencoder to reduce the dimensionality of the data.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 09:18:03.458944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 09:18:04.015804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 09:18:04.892523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:04.919949: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:04.919996: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The data should already be present as `dataset.csv` and `top_unigrams.txt` in the `data` folder.

If they are not present, do the following.
1. Ensure that the VirusTotal reports are present in `data/json` with the format `[LABEL]_[HASH].json`.
3. Run `prepare_data.py`. This will generate the two files needed for this notebook.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,AAUTO,c35885d8463d1fe937cf5afb628e5f37ac2c33004b90da...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AENJARIS,6eba466355df18050554910e3aece28ac7118d6f9683a2...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AENJARIS,a463230d154886983071433608b97630644aeb46fd2a6e...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AGENTB,0022508fd02bb23c3a2c4f5de0906df506a2fcabc3e841...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AGENTB,08174ddc79fd17dee63232b6aa50c79b96d512546aead8...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,ZAPCHAST,04d6ff264286ef70cdc08e69cebc09a6cfd2752e5ba3a1...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
981,ZAPCHAST,a368fa01248ecb84c56c87fe65edde7f7d3730e1a37e87...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
982,ZAPCHAST,dfce96433887553201a295b1475373a82ac6a730f18d88...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
983,ZEGOST,1367ecca54ac27ce18179d6bfcc0ff93bb7cfb2882dc60...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the training of the model, we don't need the label or the file hash.

In [5]:
df = raw_df.drop(columns=["label", "hash"])
df

,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,dim-0009,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


80% of the dataframe will be saved for training, while 20% will be left for testing.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=42
)

In [7]:
X_train.shape

(788, 10000)

In [8]:
X_test.shape

(197, 10000)

# Model Creation

We will use an autoencoder to reduce the dimensionality of the data.

In [9]:
from keras import layers
from keras.models import Sequential

In [10]:
LAYER_SIZES = [2048, 512, 128, 32]  # The last layer is the center layer

In [11]:
INITIAL_EPOCH = 0

def create_encoder():
    model = Sequential(name="Encoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))

    for layer_size in LAYER_SIZES[:-1]:
        model.add(layers.Dense(layer_size, activation="relu"))

    # Add an activity regularizer to make the middle layer sparse
    model.add(layers.Dense(LAYER_SIZES[-1], activation="relu"))
    
    return model


def create_decoder():
    model = Sequential(name="Decoder")
    model.add(keras.Input((LAYER_SIZES[-1],), name="decoder-input"))

    for layer_size in LAYER_SIZES[-2::-1]:  # Starting from second last
        model.add(layers.Dense(layer_size, activation="relu"))
    model.add(layers.Dense(df.shape[1], activation="relu"))
    
    return model


def create_autoencoder(encoder, decoder):
    model = Sequential(name="Autoencoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))
    model.add(encoder)
    model.add(decoder)

    model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"]
    )

    return model

In [12]:
encoder = create_encoder()
decoder = create_decoder()
autoencoder = create_autoencoder(encoder, decoder)

2024-04-29 09:18:06.099695: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:06.099796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:06.099830: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:06.884910: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 09:18:06.884980: I external/local_xla/xla/stream_executor

In [13]:
autoencoder.summary(expand_nested=True)

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │    21,600,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 10000)          │    21,610,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_4 (Dense)            │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_5 (Dense)            │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 10000)          │    20,490,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,211,824 (164.84 MB)

 Trainable params: 43,211,824 (164.84 MB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [14]:
import os

checkpoint_path = "../models/encoder/checkpoints/{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Define some utility functions that help identify the best model trained so far.

In [16]:
def find_latest_model(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)
    dir_contents = os.listdir(checkpoint_dir)
    if len(dir_contents) == 0:
        return None
    latest_model_path = sorted(dir_contents)[-1]
    return latest_model_path


def load_latest_model(checkpoint_dir):
    latest_model_path = find_latest_model(checkpoint_dir)
    if latest_model_path is None:
        print("No checkpoints found, not loading anything")
        return None
    print(f"Loading '{latest_model_path}'")
    model = keras.models.load_model(os.path.join(checkpoint_dir, latest_model_path))
    print("Done")
    return model

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 17
# autoencoder = load_latest_model(checkpoint_dir)

Train the model.

In [18]:
NUM_EPOCHS = 200

autoencoder.fit(
    X_train,
    X_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/200


I0000 00:00:1714353488.826632  636102 service.cc:145] XLA service 0x7fc2cc008250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714353488.826729  636102 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 09:18:08.858375: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 09:18:09.671575: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 5/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0195 - mae: 0.0252

I0000 00:00:1714353492.427882  636102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 0.0182 - mae: 0.0267
Epoch 1: val_loss improved from inf to 0.01500, saving model to ../models/encoder/checkpoints/0001.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 293ms/step - loss: 0.0181 - mae: 0.0266 - val_loss: 0.0150 - val_mae: 0.0223
Epoch 2/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0144 - mae: 0.0212
Epoch 2: val_loss improved from 0.01500 to 0.01348, saving model to ../models/encoder/checkpoints/0002.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0143 - mae: 0.0211 - val_loss: 0.0135 - val_mae: 0.0198
Epoch 3/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0128 - mae: 0.0190
Epoch 3: val_loss improved from 0.01348 to 0.01185, saving model to ../models/encoder/checkpoints/0003.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0128 - mae: 0.0190 - val_loss: 0.0118 - val_mae: 0.0180
Epoch 4/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0120 - mae: 0.0178
Epoch 4: val_loss improved from 0.01185 to 0

Load the best performing model.

In [19]:
autoencoder = load_latest_model(checkpoint_dir)

Loading '0050.keras'
Done


Evaluate the autoencoder on the test data.

In [20]:
test_mse, test_mae = autoencoder.evaluate(X_test, X_test, verbose=1)
print(f"Testing MSE: {test_mse:5.5f}")
print(f"Testing MAE: {test_mae:5.5f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 0.0073 - mae: 0.0092
Testing MSE: 0.00796
Testing MAE: 0.01010


Get only the encoder part to save.

In [21]:
encoder = autoencoder.get_layer("Encoder")

In [22]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,600,928 (82.40 MB)

 Trainable params: 21,600,928 (82.40 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
encoder.save("../models/encoder/encoder.keras")

# Transforming Original Data

In [24]:
transformed_df = encoder.predict(df)

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [25]:
transformed_df = pd.DataFrame(transformed_df, columns=[f"dim-{i:02d}" for i in range(LAYER_SIZES[-1])])

Add the labels and hashes back to the dataframe.

In [26]:
transformed_df.insert(0, "label", raw_df["label"])
transformed_df.insert(1, "hash", raw_df["hash"])

In [27]:
transformed_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,AAUTO,c35885d8463d1fe937cf5afb628e5f37ac2c33004b90da...,0.46469,0.0,0.0,11.658468,0.0,2.977732,0.0,7.066464,...,0.000000,17.847954,0.0,0.000000,0.0,3.958649,5.503294,0.061921,5.570132,6.377898
1,AENJARIS,6eba466355df18050554910e3aece28ac7118d6f9683a2...,0.00000,0.0,0.0,12.925267,0.0,18.161427,0.0,10.700646,...,0.000000,25.555328,0.0,0.000000,0.0,0.000000,0.000000,10.576216,0.442560,25.964514
2,AENJARIS,a463230d154886983071433608b97630644aeb46fd2a6e...,0.00000,0.0,0.0,13.213775,0.0,18.369963,0.0,10.656313,...,0.000000,25.844568,0.0,0.000000,0.0,0.000000,0.000000,10.629220,0.955844,26.305899
3,AGENTB,0022508fd02bb23c3a2c4f5de0906df506a2fcabc3e841...,0.00000,0.0,0.0,14.689069,0.0,6.657124,0.0,0.000000,...,0.000000,12.291744,0.0,5.291505,0.0,0.000000,0.000000,0.000000,21.850214,9.870970
4,AGENTB,08174ddc79fd17dee63232b6aa50c79b96d512546aead8...,0.00000,0.0,0.0,0.000000,0.0,1.135266,0.0,0.000000,...,0.000000,7.235934,0.0,0.000000,0.0,0.000000,0.901341,0.000000,3.588072,7.317095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,ZAPCHAST,04d6ff264286ef70cdc08e69cebc09a6cfd2752e5ba3a1...,0.00000,0.0,0.0,2.240609,0.0,13.727724,0.0,0.000000,...,3.688204,16.599146,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,6.045621
981,ZAPCHAST,a368fa01248ecb84c56c87fe65edde7f7d3730e1a37e87...,0.00000,0.0,0.0,5.219143,0.0,12.790781,0.0,0.000000,...,0.000000,10.824963,0.0,2.004188,0.0,0.000000,0.000000,0.000000,9.681213,5.848680
982,ZAPCHAST,dfce96433887553201a295b1475373a82ac6a730f18d88...,0.00000,0.0,0.0,0.000000,0.0,10.820048,0.0,0.000000,...,0.000000,9.915873,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.068173,14.177855
983,ZEGOST,1367ecca54ac27ce18179d6bfcc0ff93bb7cfb2882dc60...,0.00000,0.0,0.0,0.000000,0.0,1.703352,0.0,0.364429,...,0.000000,13.974449,0.0,0.000000,0.0,0.000000,12.804555,0.000000,2.894542,26.230101


In [28]:
transformed_df.to_csv("../data/encoded-data.csv", index=False)